### CSIRO - Image2Biomass Prediction
URL: https://www.kaggle.com/competitions/csiro-biomass/overview

In [1]:
import pandas as pd
import numpy as np

print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

NameError: name 'torch' is not defined

In [ ]:
# data import
train_df = pd.read_csv('/kaggle/input/csiro-biomass/train.csv')
test_df = pd.read_csv('/kaggle/input/csiro-biomass/test.csv')
sample_submission = pd.read_csv('/kaggle/input/csiro-biomass/sample_submission.csv')
train_img = torch.utils.data.DataLoader('/kaggle/input/csiro-biomass/train')
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
train_df

In [ ]:
# extract target columns
y_train = torch.tensor(train_df['target'].values.astype(np.float32))
y_train = y_train.unsqueeze(1)

y_train

In [ ]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image

class ImageFeatureExtractor:
    def __init__(self, model_name='resnet50'):
        # select model
        if model_name == 'resnet50':
            model = models.resnet50(pretrained=True)
            # remove last layer
            self.encoder = torch.nn.Sequential(*list(model.children())[:-1])
            self.feature_dim = 2048

        elif model_name == 'resnet18':
            model = models.resnet18(pretrained=True)
            self.encoder = torch.nn.Sequential(*list(model.children())[:-1])
            self.feature_dim = 512

        elif model_name == 'efficientnet_b0':
            model = models.efficientnet_b0(pretrained=True)
            self.encoder = model.features
            self.encoderl.add_module('avgpool', torch.nn.AdaptiveAvgPool2d(1))
            self.feature_dim = 1280

        self.encoder.eval()

        # preprocessing
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def extract_features(self, image_path):
        # load image
        image = Image.open(image_path).convert('RGB')

        # preprocessing
        input_tensor = self.transform(image).unsqueeze(0)

        # extrach feature
        with torch.no_grad():
            features = self.encoder(input_tensor)

        # flatten and numpy
        features = features.squeeze().numpy()

        return features

    def extract_features_dict(self, image_path, prefix='img'):
        # return dict
        features = self.extract_features(image_path)

        # convert dict
        features_dict = {
            f'{prefix}_feat_{i}': float(val)
            for i, val in enumerate(features)
        }

        return features_dict

